## Solving bidomain equations on a slice of the heart

### Importing required libraries

In [1]:
%cd ../../..
from src.models.dynamics_models import *
from src.models.cell_models import *
from src.utils import *
import ufl

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-tkiva


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = heart_slice()
x = ufl.SpatialCoordinate(domain)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 40%] Meshing curve 2 (Nurb)
Info    : [ 70%] Meshing curve 3 (Nurb)
Info    : Done meshing 1D (Wall 0.0249705s, CPU 0.025428s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0641932s, CPU 0.064082s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 2.2691e-05s, CPU 2.2e-05s)
Info    : 3628 nodes 7231 elements


In [3]:
class CellModel(BeelerReuter):
    def apply_current(self):
        locator1 = lambda x: (x[0] - 0) ** 2 + (x[1] - 2.1) ** 2 < 0.7**2
        locator2 = lambda x: (x[0] - 1.2) ** 2 + (x[1] - 1.5) ** 2 < 0.7**2
        locator3 = lambda x: (x[0] - 3.3) ** 2 + (x[1] - 3.1) ** 2 < 0.3**2
        cells1 = fem.locate_dofs_geometrical(self.V1, locator1)
        cells2 = fem.locate_dofs_geometrical(self.V1, locator2)
        cells3 = fem.locate_dofs_geometrical(self.V1, locator3)
        self.I_app.x.array[cells1] = np.full_like(cells1, 10)
        self.I_app.x.array[cells2] = np.full_like(cells2, 10)
        self.I_app.x.array[cells3] = np.full_like(cells3, 10)
        self.I_app_duration = 20

In [4]:
class Model(BidomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = -85

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector([
            -(x[1] - y_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
            (x[0] - x_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        ])

        # Healthy conductivities
        self.M_i = self.SIGMA_IT * ufl.Identity(2) + (
            self.SIGMA_IL - self.SIGMA_IT
        ) * ufl.outer(self.fibers, self.fibers)
        self.M_e = self.SIGMA_ET * ufl.Identity(2) + (
            self.SIGMA_EL - self.SIGMA_ET
        ) * ufl.outer(self.fibers, self.fibers)

In [5]:
cell_model = CellModel(domain)
model = Model(domain, cell_model)

TypeError: 'function' object is not subscriptable

In [ ]:
cell_model.plot_I_app(show_mesh=False)

### Solving equations with given parameters

In [ ]:
model.solve(
    T=400,
    steps=10000,
    save_to="slice/bidomain_BR.mp4",
    signal_point=[1.0, 3.0, 0.0],
    camera_direction="xy",
    cmap="jet",
)

### Plotting a fiber orientations

In [ ]:
x_c, y_c = 0.2, 1
plot_vector_field(
    domain,
    lambda x: (
        -(x[1] - y_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        (x[0] - x_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ),
    0.03,
    0.7,
)

### Plotting a final plot of transmembrane potential

In [ ]:
plot_function(model.V_m_n, "V_m", show_mesh=False)

### Plotting a transmembrane potential in a given signal point

In [ ]:
model.plot_signal()